In [26]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.models import save_model

In [2]:
# Load input data into a DataFrame
df = pd.read_csv('C:/Workspace/lstm-model-data/91-Site_1A-Trina.csv')
df

,timestamp,Active_Energy_Delivered_Received,Current_Phase_Average,Active_Power,Wind_Speed,Weather_Temperature_Celsius,Weather_Relative_Humidity,Global_Horizontal_Radiation,Diffuse_Horizontal_Radiation,Wind_Direction,Weather_Daily_Rainfall,Radiation_Global_Tilted,Radiation_Diffuse_Tilted
0,8/14/2013 15:35,113970.0,11.464333,6.208334,3.100190,28.226471,17.152653,526.585205,54.478363,179.385696,0.0,661.085877,61.034492
1,8/14/2013 15:40,113971.0,11.152334,6.106200,3.415439,27.399916,17.834187,512.387695,55.947868,146.240540,0.0,641.358826,58.497375
2,8/14/2013 15:45,113971.0,10.898998,6.010433,2.373577,27.661219,17.747263,493.852539,51.699211,191.351410,0.0,623.269104,57.587238
3,8/14/2013 15:50,113972.0,10.557001,5.924666,3.515019,27.724470,17.451172,475.453247,48.139847,179.864182,0.0,604.026062,56.896786
4,8/14/2013 15:55,113972.0,10.284998,5.852333,3.287408,27.314009,17.803726,463.113190,51.068470,160.973099,0.0,584.602722,56.105083
...,...,...,...,...,...,...,...,...,...,...,...,...,...
982545,1/19/2023 11:10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,981.544312,99.513496
982546,1/19/2023 11:15,294840.0,14.935999,7.162267,NaN,30.059155,35.795490,1066.155518,105.059143,46.717766,0.0,1000.187561,101.900909
982547,1/19/2023 11:20,294841.0,15.017334,7.184267,NaN,30.250393,36.004295,1072.618530,103.956429,46.688675,0.0,1011.221436,99.269295
982548,1/19/2023 11:25,294841.0,15.113668,7.393467,NaN,30.581776,34.828823,1082.490112,105.972069,46.601555,0.0,NaN,NaN


In [3]:
# Convert timestamp column to datetime
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Extract hour, minute, and second
df['Time_Format'] = df['timestamp'].dt.strftime('%H:%M:%S')

# Convert time_format column to timedelta
df['Time_Delta'] = pd.to_timedelta(df['Time_Format'])

# Calculate time in milliseconds
df['Time_Ms'] = df['Time_Delta'].dt.total_seconds() * 1000

df

,timestamp,Active_Energy_Delivered_Received,Current_Phase_Average,Active_Power,Wind_Speed,Weather_Temperature_Celsius,Weather_Relative_Humidity,Global_Horizontal_Radiation,Diffuse_Horizontal_Radiation,Wind_Direction,Weather_Daily_Rainfall,Radiation_Global_Tilted,Radiation_Diffuse_Tilted,Time_Format,Time_Delta,Time_Ms
0,2013-08-14 15:35:00,113970.0,11.464333,6.208334,3.100190,28.226471,17.152653,526.585205,54.478363,179.385696,0.0,661.085877,61.034492,15:35:00,0 days 15:35:00,56100000.0
1,2013-08-14 15:40:00,113971.0,11.152334,6.106200,3.415439,27.399916,17.834187,512.387695,55.947868,146.240540,0.0,641.358826,58.497375,15:40:00,0 days 15:40:00,56400000.0
2,2013-08-14 15:45:00,113971.0,10.898998,6.010433,2.373577,27.661219,17.747263,493.852539,51.699211,191.351410,0.0,623.269104,57.587238,15:45:00,0 days 15:45:00,56700000.0
3,2013-08-14 15:50:00,113972.0,10.557001,5.924666,3.515019,27.724470,17.451172,475.453247,48.139847,179.864182,0.0,604.026062,56.896786,15:50:00,0 days 15:50:00,57000000.0
4,2013-08-14 15:55:00,113972.0,10.284998,5.852333,3.287408,27.314009,17.803726,463.113190,51.068470,160.973099,0.0,584.602722,56.105083,15:55:00,0 days 15:55:00,57300000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
982545,2023-01-19 11:10:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,981.544312,99.513496,11:10:00,0 days 11:10:00,40200000.0
982546,2023-01-19 11:15:00,294840.0,14.935999,7.162267,NaN,30.059155,35.795490,1066.155518,105.059143,46.717766,0.0,1000.187561,101.900909,11:15:00,0 days 11:15:00,40500000.0
982547,2023-01-19 11:20:00,294841.0,15.017334,7.184267,NaN,30.250393,36.004295,1072.618530,103.956429,46.688675,0.0,1011.221436,99.269295,11:20:00,0 days 11:20:00,40800000.0
982548,2023-01-19 11:25:00,294841.0,15.113668,7.393467,NaN,30.581776,34.828823,1082.490112,105.972069,46.601555,0.0,NaN,NaN,11:25:00,0 days 11:25:00,41100000.0


In [4]:
# Select feature columns
df_cols = ['Time_Ms', 'Weather_Temperature_Celsius', 'Active_Power']
df = df[df_cols]

df

,Time_Ms,Weather_Temperature_Celsius,Active_Power
0,56100000.0,28.226471,6.208334
1,56400000.0,27.399916,6.106200
2,56700000.0,27.661219,6.010433
3,57000000.0,27.724470,5.924666
4,57300000.0,27.314009,5.852333
...,...,...,...
982545,40200000.0,NaN,NaN
982546,40500000.0,30.059155,7.162267
982547,40800000.0,30.250393,7.184267
982548,41100000.0,30.581776,7.393467


In [5]:
# Check missing data
df.isna().sum(axis=0)

Time_Ms                            0
Weather_Temperature_Celsius    16629
Active_Power                   16723
dtype: int64

In [6]:
# Drop missing data
df = df.dropna()
df.isna().sum(axis=0)

Time_Ms                        0
Weather_Temperature_Celsius    0
Active_Power                   0
dtype: int64

In [7]:
# Extract input features and target variable
X = df[['Time_Ms', 'Weather_Temperature_Celsius']].values
y = df['Active_Power'].values

In [8]:
X

array([[5.61000000e+07, 2.82264709e+01],
       [5.64000000e+07, 2.73999157e+01],
       [5.67000000e+07, 2.76612186e+01],
       ...,
       [4.08000000e+07, 3.02503929e+01],
       [4.11000000e+07, 3.05817757e+01],
       [4.14000000e+07, 3.02931538e+01]])

In [9]:
y

array([6.20833397, 6.10619974, 6.0104332 , ..., 7.18426657, 7.39346743,
       7.3293333 ])

In [10]:
# Normalize the input features and target variable
scaler_X = MinMaxScaler(feature_range=(0, 1))
X_scaled = scaler_X.fit_transform(X)

scaler_y = MinMaxScaler(feature_range=(0, 1))
y_scaled = scaler_y.fit_transform(y.reshape(-1, 1))

In [11]:
X_scaled

array([[0.65156794, 0.66934962],
       [0.65505226, 0.6612391 ],
       [0.65853659, 0.66380312],
       ...,
       [0.4738676 , 0.68920922],
       [0.47735192, 0.69246089],
       [0.48083624, 0.68962881]])

In [12]:
y_scaled

array([[0.54954694],
       [0.54056605],
       [0.53214509],
       ...,
       [0.63536283],
       [0.65375832],
       [0.64811887]])

In [13]:
# Reshape the input features to fit the LSTM input shape
timesteps = 1  # Number of time steps (1 in this case)
input_dim = X_scaled.shape[1]  # Number of input features

X_scaled = X_scaled.reshape((X_scaled.shape[0], timesteps, input_dim))

In [14]:
X_scaled

array([[[0.65156794, 0.66934962]],

       [[0.65505226, 0.6612391 ]],

       [[0.65853659, 0.66380312]],

       ...,

       [[0.4738676 , 0.68920922]],

       [[0.47735192, 0.69246089]],

       [[0.48083624, 0.68962881]]])

In [17]:
# Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

In [18]:
# Define the LSTM model with scaled output
model = Sequential()
model.add(LSTM(64, input_shape=(timesteps, input_dim)))
model.add(Dense(1, activation='sigmoid'))  # Sigmoid activation for scaled output

In [19]:
# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam')

In [20]:
# Train the LSTM model
model.fit(X_train, y_train, epochs=10, batch_size=32)

Epoch 1/10
24146/24146 [==============================] - 36s 1ms/step - loss: 0.0173
Epoch 2/10
24146/24146 [==============================] - 33s 1ms/step - loss: 0.0121
Epoch 3/10
24146/24146 [==============================] - 33s 1ms/step - loss: 0.0120
Epoch 4/10
24146/24146 [==============================] - 34s 1ms/step - loss: 0.0119
Epoch 5/10
24146/24146 [==============================] - 33s 1ms/step - loss: 0.0118
Epoch 6/10
24146/24146 [==============================] - 32s 1ms/step - loss: 0.0117
Epoch 7/10
24146/24146 [==============================] - 32s 1ms/step - loss: 0.0117
Epoch 8/10
24146/24146 [==============================] - 33s 1ms/step - loss: 0.0116
Epoch 9/10
24146/24146 [==============================] - 33s 1ms/step - loss: 0.0116
Epoch 10/10
24146/24146 [==============================] - 34s 1ms/step - loss: 0.0115


In [21]:
# Evaluate the model on the test set
mse = model.evaluate(X_test, y_test)
mse

6037/6037 [==============================] - 6s 900us/step - loss: 0.0116


0.011569103226065636

In [22]:
# Make predictions on the test set
predictions_scaled = model.predict(X_test)
predictions_scaled

6037/6037 [==============================] - 6s 855us/step


array([[5.5294305e-01],
       [2.8506044e-04],
       [1.0019137e-02],
       ...,
       [4.5090857e-01],
       [2.3029097e-03],
       [5.5636640e-04]], dtype=float32)

In [23]:
# Rescale the predictions back to the original range
predictions_rescaled = scaler_y.inverse_transform(predictions_scaled)
predictions_rescaled

array([[ 6.2469563 ],
       [-0.03809151],
       [ 0.07260829],
       ...,
       [ 5.086579  ],
       [-0.01514372],
       [-0.03500611]], dtype=float32)

In [24]:
predictions_scaled = model.predict(X_scaled)
predictions_scaled

30182/30182 [==============================] - 25s 837us/step


array([[0.39920035],
       [0.38590485],
       [0.3793362 ],
       ...,
       [0.548562  ],
       [0.55190986],
       [0.5524849 ]], dtype=float32)

In [25]:
predictions_rescaled = scaler_y.inverse_transform(predictions_scaled)
predictions_rescaled

array([[4.4985323],
       [4.3473306],
       [4.2726293],
       ...,
       [6.197133 ],
       [6.235206 ],
       [6.2417464]], dtype=float32)

In [27]:
save_model(model, '../lstm-model.h5')